In [2]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import time

In [2]:
# ==========================================
# 1️⃣ เชื่อมต่อ Cassandra
# ==========================================
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS data_stock
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('data_stock')

session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol text,
        time timestamp,
        open float,
        high float,
        low float,
        close float,
        volume bigint,
        value float,
        PRIMARY KEY (symbol, time)
    ) WITH CLUSTERING ORDER BY (time ASC);
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")



✅ Keyspace และ Table พร้อมใช้งาน!
